<a href="https://colab.research.google.com/github/nonoumasy/twitter_sent_analysis/blob/master/twitter_sent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.5MB/s 


In [0]:
import tweepy
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [0]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [0]:
#vaderSentiment
analyzer = SentimentIntensityAnalyzer()

def get_related_tweets(keyword):
  twitter_users = []
  twitter_user_loc = []
  twitter_user_status_count = []
  twitter_user_follower_count = []
  tweet_time = []
  tweet_string = []
  
  tweet_retweet = []

   
  for tweet in tweepy.Cursor(api.search,q=keyword, count=1000).items(1000):
          if (not tweet.retweeted) and ('RT @' not in tweet.text):
              if tweet.lang == "en":
                  twitter_users.append(tweet.user.name)
                  twitter_user_loc.append(tweet.user.location)
                  twitter_user_status_count.append(tweet.user.statuses_count)
                  twitter_user_follower_count.append(tweet.user.followers_count)
                  tweet_retweet.append(tweet.retweet_count)
                  tweet_time.append(tweet.created_at)
                  tweet_string.append(tweet.text)
                  print([tweet.user.name,tweet.created_at,tweet.text])
  df = pd.DataFrame({'name' :twitter_users,
                     'location' : twitter_user_loc, 
                     'time' : tweet_time,
                     'retweet' : tweet_retweet,
                     'status_count' : twitter_user_status_count,
                     'follow_count' : twitter_user_follower_count,
                     'tweet' : tweet_string,
                     })
  
  #using VaderSentiment
  df['vader_sentiment'] = df['tweet'].apply(lambda x : analyzer.polarity_scores(x))

  #get the compound score from the VaderSentiment dict
  df['vader_comp'] = df['vader_sentiment'].apply(lambda x: x.get('compound'))

  return df

In [153]:
df = get_related_tweets('bill murray')

['Petrarca', datetime.datetime(2020, 1, 28, 20, 35, 47), '@ceg6 @JessicaRemoNJ @willmenaker "Nixon!" from Where The Buffalo Roam. Bill Murray as Hunter S. Thompson.\nhttps://t.co/CDUchH3vXJ']
['Movie Merch Mania', datetime.datetime(2020, 1, 28, 20, 23), '1984 Ghostbusters Advance 1 Sheet Movie Poster Bill Murray Dan Aykroyd Classic  https://t.co/aXEH4p5Lkw https://t.co/IzuEWFeusl']
['Alex Plante 🤓', datetime.datetime(2020, 1, 28, 20, 16, 23), "Tag yourself I'm Bill Murray's passive aggressive wife back home.\n\nGod every single thing she says is pure gold. https://t.co/LxwYpadzni"]
['The Playmaker', datetime.datetime(2020, 1, 28, 20, 4, 16), "@TyJames77 Idk. I don't think Murray has made too many bad trades. Nobody saw Pettersson or Wild Bill becoming the… https://t.co/ttb9INZR9c"]
['Curt Deppert', datetime.datetime(2020, 1, 28, 19, 50, 30), '#FYI There you go... Bill Murray returns to Woodstock Square to film a #Jeep commercial https://t.co/LdGDgkoD1m via @nwherald']
['Curt Deppert', 

In [154]:
df.head()

,name,location,time,retweet,status_count,follow_count,tweet,vader_sentiment,vader_comp
0,Petrarca,Riding Occam's Razor Scooter.,2020-01-28 20:35:47,0,6164,67,"@ceg6 @JessicaRemoNJ @willmenaker ""Nixon!"" fro...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,Movie Merch Mania,,2020-01-28 20:23:00,0,31925,214,1984 Ghostbusters Advance 1 Sheet Movie Poster...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,Alex Plante 🤓,"Wienerpeg, Manitoba",2020-01-28 20:16:23,0,3674,791,Tag yourself I'm Bill Murray's passive aggress...,"{'neg': 0.071, 'neu': 0.756, 'pos': 0.173, 'co...",0.3182
3,The Playmaker,"Orange County, CA",2020-01-28 20:04:16,0,26936,421,@TyJames77 Idk. I don't think Murray has made ...,"{'neg': 0.214, 'neu': 0.786, 'pos': 0.0, 'comp...",-0.5994
4,Curt Deppert,In the State of Illinois,2020-01-28 19:50:30,0,38509,670,#FYI There you go... Bill Murray returns to Wo...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


In [155]:
#summary
vader_pos = df[df['vader_comp'] > 0.0]
vader_neg = df[df['vader_comp'] < 0.0]

print("Number of Positive Tweets", len(vader_pos))
print("Number of Negative Tweets", len(vader_neg))
print("Number of Neutral Tweets", df.shape[0] - (len(vader_neg) + len(vader_pos)))

Number of Positive Tweets 150
Number of Negative Tweets 53
Number of Neutral Tweets 145


In [156]:
#negative tweets
df[df['vader_comp'] < 0.0]

,name,location,time,retweet,status_count,follow_count,tweet,vader_sentiment,vader_comp
3,The Playmaker,"Orange County, CA",2020-01-28 20:04:16,0,26936,421,@TyJames77 Idk. I don't think Murray has made ...,"{'neg': 0.214, 'neu': 0.786, 'pos': 0.0, 'comp...",-0.5994
7,IanHeathen,High Desert - Low Expectations,2020-01-28 19:36:26,0,94,22,@nachosarah I was just talking about this toda...,"{'neg': 0.292, 'neu': 0.708, 'pos': 0.0, 'comp...",-0.6800
8,RCJM,,2020-01-28 19:27:10,0,99,1,@JaneyGodley @OnlyAnExcuse Is that Bill Murray...,"{'neg': 0.213, 'neu': 0.787, 'pos': 0.0, 'comp...",-0.2263
10,James Duncan Comics,"Montréal, Québec",2020-01-28 19:20:29,0,5034,415,@nachosarah There will never be anyone to repl...,"{'neg': 0.178, 'neu': 0.822, 'pos': 0.0, 'comp...",-0.5697
12,Stadium Goods,New York City,2020-01-28 19:14:49,2,6898,92299,Bill Murray saves the day in the Air Jordan 2 ...,"{'neg': 0.114, 'neu': 0.886, 'pos': 0.0, 'comp...",-0.4019
16,00101010,"Portland, OR",2020-01-28 18:58:20,0,2736,221,@nachosarah Do you want a dead Bill Murray? Ca...,"{'neg': 0.448, 'neu': 0.505, 'pos': 0.047, 'co...",-0.9203
21,Classic Film & TV,,2020-01-28 18:40:08,0,119,53,"Why Bill Murray Hated the Movie ""Groundhog Day...","{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp...",-0.6369
24,TJD2,United States,2020-01-28 18:35:15,0,32,7,@nachosarah So wait... is Bill Murray dead now...,"{'neg': 0.566, 'neu': 0.434, 'pos': 0.0, 'comp...",-0.8784
26,cooper,North Korea,2020-01-28 18:31:12,0,22894,480,what is going on with my dreams lately. i just...,"{'neg': 0.136, 'neu': 0.746, 'pos': 0.118, 'co...",-0.1027
33,TERESA BOWIEMEGAFAN🖋🇬🇧⚡⭐,#davidbowieglamourfanzine,2020-01-28 18:11:09,0,54243,2223,@Brookranger @carrie_patino @khelm74 @jill_m_s...,"{'neg': 0.214, 'neu': 0.627, 'pos': 0.159, 'co...",-0.2428
